**WEATHER AGENT**


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import json

load_dotenv()

client = OpenAI()

In [3]:
def get_weather(city: str):
    """Simulates weather API call"""
    # TODO: Replace with actual API call
    weather_data = {
        "New York": "28°C, Partly Cloudy",
        "London": "15°C, Rainy",
        "Tokyo": "22°C, Sunny",
    }
    return weather_data.get(city, f"Weather data unavailable for {city}")

In [4]:
available_tools = {
    "get_weather": {
        "fn": get_weather,
        "description": "Takes a city name as input and returns current weather",
        "parameters": {"city": {"type": "string", "description": "Name of the city"}},
    }
}

In [5]:
system_prompt = """
You are a helpful AI assistant that solves user queries using structured reasoning and controlled tool use.

You MUST strictly follow the PAOR loop:
PLAN → ACTION → OBSERVE → RESPOND

════════════════════════════════════
CORE LOOP: PAOR
════════════════════════════════════

1️⃣ PLAN  
Analyze the user query and decide what to do next.  
Do NOT call any tools in this step.

Format (valid JSON only):
{
  "step": "plan",
  "content": "Your reasoning and decision-making process",
  "next_action": "tool_name | respond"
}

════════════════════════════════════

2️⃣ ACTION  
Call exactly ONE tool.

Format (valid JSON only):
{
  "step": "action",
  "tool": "tool_name",
  "parameters": {"city": "CityName"},
  "content": "Why this tool is necessary"
}

════════════════════════════════════

3️⃣ OBSERVE  
Interpret the tool result and decide what to do next.

Format (valid JSON only):
{
  "step": "observe",
  "content": "Clear summary of the tool output",
  "next_step": "continue | respond"
}

════════════════════════════════════

4️⃣ RESPOND  
Provide the final user-facing answer.

Format (valid JSON only):
{
  "step": "respond",
  "content": "Final answer to the user",
  "confidence": "high | medium | low"
}

════════════════════════════════════
AVAILABLE TOOLS
════════════════════════════════════
- get_weather(city: str): Retrieves current weather for a city

════════════════════════════════════
RULES
════════════════════════════════════
- Every response MUST be valid JSON
- Always start with PLAN step
- Maximum 10 steps per query
- Do NOT invent tools
"""

In [6]:
messages = [{"role": "system", "content": system_prompt}]

while True:
    user_query = input("\n💬 Ask (or type 'exit'): ")

    if user_query.lower() == "exit":
        print("👋 Goodbye!")
        break

    messages.append({"role": "user", "content": user_query})

    step_count = 0
    max_steps = 10

    while step_count < max_steps:
        step_count += 1

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={"type": "json_object"},
            messages=messages,
        )

        raw_response = response.choices[0].message.content
        parsed_response = json.loads(raw_response)

        messages.append({"role": "assistant", "content": raw_response})

        step = parsed_response.get("step")
        print(
            f"\n[{step_count}] {step.upper()}: {parsed_response.get('content', '')[:100]}..."
        )

        # ───────── PLAN ─────────
        if step == "plan":
            next_action = parsed_response.get("next_action")
            if next_action == "respond":
                # No tool needed, skip to respond
                continue
            # Otherwise, agent will proceed to ACTION in next iteration

            # ───────── ACTION ─────────
        elif step == "action":
            tool_name = parsed_response.get("tool")
            tool_params = parsed_response.get("parameters", {})

            if tool_name in available_tools:
                try:
                    # Extract the actual parameter (e.g., city name)
                    tool_fn = available_tools[tool_name]["fn"]

                    # Call the tool with correct parameters
                    result = tool_fn(**tool_params)  # ✅ Unpack parameters properly

                    # Add observation to context
                    messages.append(
                        {
                            "role": "assistant",
                            "content": json.dumps(
                                {
                                    "step": "observe",
                                    "content": f"Tool returned: {result}",
                                    "next_step": "continue",
                                }
                            ),
                        }
                    )

                except Exception as e:
                    messages.append(
                        {
                            "role": "assistant",
                            "content": json.dumps(
                                {
                                    "step": "observe",
                                    "content": f"Tool failed: {str(e)}",
                                    "next_step": "respond",
                                }
                            ),
                        }
                    )
            else:
                messages.append(
                    {
                        "role": "assistant",
                        "content": json.dumps(
                            {
                                "step": "observe",
                                "content": f"Tool '{tool_name}' not found",
                                "next_step": "respond",
                            }
                        ),
                    }
                )

        # ───────── OBSERVE ─────────
        elif step == "observe":
            next_step = parsed_response.get("next_step")
            if next_step == "respond":
                # Agent will generate final response in next iteration
                continue

        # ───────── RESPOND ─────────
        elif step == "respond":
            print(f"\n🤖 ASSISTANT: {parsed_response.get('content')}")
            break  # Exit agent loop

    if step_count >= max_steps:
        print("\n⚠️ Maximum steps reached. Stopping.")


[1] PLAN: The user is asking for the current weather in New York City (NYC). I need to retrieve this informati...

[2] ACTION: I am calling this tool to obtain the current weather data for New York City....

[3] RESPOND: I'm currently unable to retrieve the weather data for New York City. Please try again later....

🤖 ASSISTANT: I'm currently unable to retrieve the weather data for New York City. Please try again later.
👋 Goodbye!
